NAME : Pratik Vijay Bhagyawant\ 
CLASS : TE-AI&DS\
ROLL No : 2437016\
Batch : A\
SUB : Software Laboratory II (Artificial Neural Network)

Problem Statement :\
Create a Neural network architecture from scratch in Python and use it to do multi-class
classification on any data.\
Parameters to be considered while creating the neural network from scratch are specified as:\
(1) No of hidden layers : 1 or more\
(2) No. of neurons in hidden layer: 100\
(3) Non-linearity in the layer : Relu\
(4) Use more than 1 neuron in the output layer. Use a suitable threshold value\
Use appropriate Optimisation algorithm 

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# Load and preprocess Iris dataset
iris = load_iris()
X = iris.data  # Shape: (150, 4)
y = iris.target.reshape(-1, 1)  # Shape: (150, 1)

In [3]:
# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [5]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:


# Activation functions and derivatives
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp = np.exp(x - np.max(x, axis=1, keepdims=True))  # stability fix
    return exp / np.sum(exp, axis=1, keepdims=True)

# Cross-entropy loss
def cross_entropy_loss(y_pred, y_true):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))

# Accuracy
def accuracy(y_pred, y_true):
    return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

# Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        self.lr = lr
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.bias1 = np.zeros((1, hidden_size))
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.bias2 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.weights1) + self.bias1
        self.a1 = relu(self.z1)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.a2 = softmax(self.z2)
        return self.a2

    def backward(self, X, y, y_pred):
        m = y.shape[0]
        dz2 = y_pred - y
        dw2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m

        dz1 = np.dot(dz2, self.weights2.T) * relu_derivative(self.z1)
        dw1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m

        # Update weights
        self.weights1 -= self.lr * dw1
        self.bias1 -= self.lr * db1
        self.weights2 -= self.lr * dw2
        self.bias2 -= self.lr * db2

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = cross_entropy_loss(y_pred, y)
            self.backward(X, y, y_pred)

            if epoch % 100 == 0:
                acc = accuracy(y_pred, y)
                print(f"Epoch {epoch} — Loss: {loss:.4f} — Accuracy: {acc:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

# Instantiate and train
nn = NeuralNetwork(input_size=4, hidden_size=100, output_size=3, lr=0.01)
nn.train(X_train, y_train, epochs=1000)

# Test accuracy
y_test_pred = nn.forward(X_test)
print("\nTest Accuracy:", accuracy(y_test_pred, y_test))


Epoch 0 — Loss: 1.0982 — Accuracy: 0.3333
Epoch 100 — Loss: 1.0906 — Accuracy: 0.6750
Epoch 200 — Loss: 1.0702 — Accuracy: 0.6833
Epoch 300 — Loss: 1.0106 — Accuracy: 0.8250
Epoch 400 — Loss: 0.8832 — Accuracy: 0.8750
Epoch 500 — Loss: 0.7379 — Accuracy: 0.8000
Epoch 600 — Loss: 0.6291 — Accuracy: 0.7917
Epoch 700 — Loss: 0.5510 — Accuracy: 0.8083
Epoch 800 — Loss: 0.4946 — Accuracy: 0.8083
Epoch 900 — Loss: 0.4521 — Accuracy: 0.8250

Test Accuracy: 0.8333333333333334
